https://adventofcode.com/2015/day/7

In [19]:
from collections import deque

import networkx as nx

In [2]:
datafile = 'data/07.txt'

In [3]:
with open(datafile) as fh:
    data = [y for y in (x.strip() for x in fh) if y]

In [4]:
len(data), data[:3]

(339, ['af AND ah -> ai', 'NOT lk -> ll', 'hz RSHIFT 1 -> is'])

In [91]:
def parse_line(line):
    expr, node = (x.strip() for x in line.split('->'))

    if expr.startswith('NOT'):
        op, arg = expr.split()
    elif ' AND ' in expr or ' OR ' in expr:
        arg1, op, arg2 = expr.split()
        arg = [arg1, arg2]
    elif 'SHIFT' in expr:
        arg, op, arg2 = expr.split()
        op = op + ' ' + arg2
    else:
        op = 'ASSIGN'
        arg = expr
    
    return (op, arg, node)
    

In [92]:
parse_line(data[0])

('AND', ['af', 'ah'], 'ai')

In [156]:
def build_network(data):
    g = nx.DiGraph()
    for line in data:
        op, arg, node = parse_line(line)
        g.add_node(node)
        D = g.nodes[node]
        D['op'] = op
        if op == 'ASSIGN':
            if arg.isdigit():
                D['val'] = int(arg)
            else:
                g.add_edge(arg, node)
        elif op == 'NOT' or 'SHIFT' in op:
            g.add_edge(arg, node)
        elif op in ('AND', 'OR'):
            for a in arg:
                g.add_edge(a, node)
        else:
            raise ValueError("Unrecognized op: %s", op)
    
    for node in g:
        if node.isdigit():
            g.nodes[node]['val'] = int(node)
    
    return g
        

In [157]:
g = build_network(data)

In [158]:
g.nodes['le']

{'op': 'LSHIFT 15'}

In [160]:
list(g.predecessors('le'))

['la']

In [161]:
def evaluate(node, g):
    D = g.nodes[node]
    if D.get('val') is not None:
        return True
    args = [g.nodes[x].get('val') for x in g.predecessors(node)]
    if None in args:
        return False
    op = D['op']
    if op == 'ASSIGN':
        val = args[0]
    elif op == 'NOT':
        val = ~ args[0]
    elif op == 'AND':
        val = args[0] & args[1]
    elif op == 'OR':
        val = args[0] | args[1]
    elif 'SHIFT' in op:
        shift, n = op.split()
        n = int(n)
        if shift == 'LSHIFT':
            val = args[0] << n
        else:
            val = args[0] >> n
    else:
        raise ValueError("Unknown command: %s", op)
    D['val'] = val
    return True      

In [178]:
g = build_network(data)

predset = set()
preds = ['a']
while preds:
    p = preds.pop()
    if p in predset:
        continue
    predset.add(p)
    preds.extend(g.predecessors(p))

evaled = set()
completed = set()
q = deque(node for node in predset if g.nodes[node].get('val') is not None)

In [179]:
%%time
while q:
    node = q.popleft()
    if node in completed:
        continue
    succs = [x for x in g.successors(node) if x not in evaled and x in predset]
    badsuccs = 0
    for succ in succs:
        if evaluate(succ, g):
            evaled.add(succ)
            q.append(succ)
        else:
            badsuccs += 1
    if badsuccs:
        q.append(node)
    else:
        completed.add(node)
if 'a' in evaled:
    print('a =', g.nodes[node]['val'])

a = 956
CPU times: user 11 ms, sys: 2 µs, total: 11 ms
Wall time: 10.6 ms


## part 2

In [180]:
g = build_network(data)

g.nodes['b']['val'] = 956

predset = set()
preds = ['a']
while preds:
    p = preds.pop()
    if p in predset:
        continue
    predset.add(p)
    preds.extend(g.predecessors(p))

evaled = set()
completed = set()
q = deque(node for node in predset if g.nodes[node].get('val') is not None)

In [181]:
%%time
while q:
    node = q.popleft()
    if node in completed:
        continue
    succs = [x for x in g.successors(node) if x not in evaled and x in predset]
    badsuccs = 0
    for succ in succs:
        if evaluate(succ, g):
            evaled.add(succ)
            q.append(succ)
        else:
            badsuccs += 1
    if badsuccs:
        q.append(node)
    else:
        completed.add(node)
if 'a' in evaled:
    print('a =', g.nodes[node]['val'])

a = 40149
CPU times: user 7.07 ms, sys: 2 µs, total: 7.08 ms
Wall time: 6.64 ms
